# Attention implementation for Dense Layer in Keras v2

reference: https://github.com/philipperemy/keras-attention-mechanism

In [1]:
import numpy as np
import keras
from keras.layers import Input, multiply

Using TensorFlow backend.


In [4]:
INPUT_DIM = 2
TIME_STEPS = 20
SINGLE_ATTENTION_VECTOR = False # if True, the attention vector is shared across the input_dimentions where the attention is applied
APPLY_ATTENTION_BEFORE_LSTM = False

In [6]:
def attention_3d_blocK(inputs):
    # inputs.shape = (batch_size, time_stamps, input_dim)
    input_dim = int(input.shape[2])
    a = Permute((2,1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, avtivations='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    return output_attention_mul